# <center>OpenAI在线大模型调用及微调方法

## <center>Ch.7 ChatCompletions模型API使用指南

### 1.Chat Completions模型概述

#### 1.1 Chat Completions模型API基本情况介绍

&emsp;&emsp;尽管在上一小节末尾，我们尝试编写了一个可以预设风格的多轮对话函数，但实际上，相比Completions模型，Chat Completions模型（对话大模型）会更适合执行对话类任务。正如上一小节所说，Chat Completions模型是一类专门围绕对话类任务进行训练和微调的模型，截至目前，OpenAI发布的Chat Completions模型主要包括gpt-3.5和gpt-4两类模型，当然，这两个模型也是目前ChatGPT应用程序背后的对话大模型。其中，根据官网给出的说明，gpt-3.5模型是基于text-davinci-003微调的模型，而后者其实是属于gpt-3模型类，并且由code-davinci-002这一基座模型经过几轮微调后训练得到，对应的模型微调关系如下所示：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202306261634213.png" alt="c604cd1f126567e50e4fc6ad2c884a3" style="zoom:33%;" />

> 这里需要补充说明的是，由于OpenAI的模型训练方法并未公开，因此官网说明中的“improvement”是否是指微调其实并不确定。

当然，gpt-4则是完全重新训练的最新一代的对话类大模型，在诸多国内外大模型评测榜单上，gpt-4也是目前（多语种）对话效果最好的一类对话类大模型。根据OpenAI7月6号的更新说明，目前gpt-4 API已针对此前支付过API费用的开发者全部开放，并且计划在7月内对全部用户开放gpt-4 API权限。鉴于此，课程中也将尽量使用gpt-4 API用于教学。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/bc0440e6f31a8e331b5eecbc402c871.png" alt="bc0440e6f31a8e331b5eecbc402c871" style="zoom:20%;" />

截至目前，OpenAI发布的Chat Completions模型如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202306241505316.png" alt="0ce263719c0b5d234ae3a6b8be1eef8" style="zoom:33%;" />

不同于Completions模型，Chat Completions模型并未开放全部的API，对于大多数模型的长文本对话模型（即标注为32k的模型），也需要填写申请方可使用，填写地址：https://platform.openai.com/docs/guides/rate-limits/overview 。而具体当前账户能够调用哪些对话类大模型的API，可以在个人中心的Rate limits页面查看：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202306261652051.png" alt="67e8aef163e8a273b5a36115a89929d" style="zoom:33%;" />

#### 1.2 2023/06/13 OpenAI API updates     

&emsp;&emsp;即然介绍Chat Completions模型，不得不提的就是OpenAI在2023年6月13日发布的以Chat Completions模型为主的重磅更新。可以说，这是自gpt-4 API公布申请之后OpenAI发布的最大的模型更新，而这些围绕着Chat Completions模型进行的更新，也将是接下来课程内容学习的重点。这里我们简单总结0613更新内容如下：

- 发布了0613编号的模型，包括gpt-3.5-turbo-0613、gpt-4-turbo-0613等，相比此前的gpt-3.5和gpt-4模型，这些0613编号模型新增了函数API的调用功能，使得其可以调用一些外部工具（例如可以调用ChatGPT插件的API）来实现某些功能。毫无疑问这是一次极具革命性的创举，在此之前，大语言模型的函数API调用只能借助LangChain来完成；新增此功能之后，围绕OpenAI大语言模型的AI应用程序开发的门槛也将大幅降低；      
- 发布了最大支持16k上下文的模型，例如gpt-3.5-turbo-16k、gpt-3.5-turbo-16k-0613等，在默认情况下gpt模型的最大上下文token限制为4k（4096个token），而16k版本的模型则将这个最大限制拓宽为16k个token（16384个token），这毫无疑问会大幅增加模型阅读长文本信息的能力和多轮对话能力；不过这里需要注意，没有标注16k的模型上下文仍然最大只支持4k；        
- 大幅降低了部分模型API的调用，其中text-embedding-ada-002模型使用费用降低了75%，而gpt-3.5-turbo使用费用降低了25%，前者是目前最先进的Embedding模型，而后者则是目前用户调用次数最多的模型。

能够看出，对话类大语言模型既是目前使用最多的一类模型、同时也是OpenAI大力维护的一类模型。接下来，我们就从具体的对话类模型调用方法入手学习，并在本节课程的末尾为大家介绍目前工业界基于对话类大模型的应用实战案例。

#### 1.3 2023/07/06 OpenAI API updates     

&emsp;&emsp;P【penAI

gpt-4 API全面可用，Chat模型微调即将上线

---

- gpt-4团队API使用方法

&emsp;&emsp;若是对于此前通过提交申请获得的gpt-4 API，则需要通过团队API页面来查看gpt-4 API情况，具体方法如下，首先，我们能够在Rate limilts页面查看当前能够使用的gpt-4模型（若申请通过的话）：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202307052112745.png" alt="222508a4fa00351b2e5e96df56d454a" style="zoom: 33%;" />

其次，由于gpt-4 API只针对组织（Organization）开放（注意，申请gpt-4 API时也是需要以组织身份申请），因此，若要使用gpt-4 API，则需要把当前账户切换成组织模型，点击个人头像即可切换：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202307052116563.png" alt="cf091130d8d745b54f842c39386d5c5" style="zoom:33%;" />

然后点击API keys，并在页面下方点击选择组织，即可使用该API调用组织内可以用的模型，并计费到组织账户：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202307052118681.png" alt="5d0550ae6d9d02accfe15c7f0993b13" style="zoom:33%;" />

这里需要注意，同一个账户下的同一个API key，是可以在个人模式和组织模式下随时切换的，如果切换成个人模式，相同的API key则会调用个人账户可以使用的模型，并在个人账户计费。而如果切换成组织模式，则可以调用组织下可以下调用的API，并且在组织账户内进行计费。

---

### 2.ChatCompletion.create API使用方法及参数解释

#### 2.1 ChatCompletion.create函数使用简例

&emsp;&emsp;接下来，我们尝试调用OpenAI的Chat类模型，并详细解释模型API参数。和调用Completion模型需要使用Completion.create函数类似，若要调用Chat类大模型，则需要使用ChatCompletion.create函数。由于Chat模型本身是基于Completion模型的、专门用于处理对话类任务的模型，因此ChatCompletion.create的使用方法和Completion.create非常类似，整体函数使用流程和核心参数都非常类似，这里我们直接对Chat模型提问“什么是机器学习？”提问和获取结果流程如下：

In [33]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [45]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "请问，什么是机器学习？"}
  ]
)

In [35]:
response

<OpenAIObject chat.completion id=chatcmpl-7aMK1Zr3sFfDrvScu0rGRShzbjNsI at 0x2e70127e980> JSON: {
  "id": "chatcmpl-7aMK1Zr3sFfDrvScu0rGRShzbjNsI",
  "object": "chat.completion",
  "created": 1688899969,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\u673a\u5668\u5b66\u4e60\u662f\u4e00\u79cd\u4eba\u5de5\u667a\u80fd\u9886\u57df\u7684\u6280\u672f\uff0c\u901a\u8fc7\u8ba9\u8ba1\u7b97\u673a\u7cfb\u7edf\u6839\u636e\u5927\u91cf\u6570\u636e\u81ea\u52a8\u5b66\u4e60\u548c\u6539\u8fdb\uff0c\u4ece\u800c\u5b9e\u73b0\u7279\u5b9a\u4efb\u52a1\u7684\u80fd\u529b\u3002\u7b80\u800c\u8a00\u4e4b\uff0c\u673a\u5668\u5b66\u4e60\u662f\u6307\u8ba1\u7b97\u673a\u7cfb\u7edf\u5229\u7528\u7edf\u8ba1\u5b66\u548c\u7b97\u6cd5\u6765\u4f7f\u81ea\u8eab\u83b7\u5f97\u4ece\u6837\u672c\u6570\u636e\u4e2d\u5b66\u4e60\u7684\u80fd\u529b\uff0c\u5e76\u5229\u7528\u5b66\u4e60\u5230\u7684\u77e5\u8bc6\u8fdb\u884c\u9884\u6d4b\u3001\u51b3\u7b56\u

能够看出，和Completion.create非常明显的一个区别在于，ChatCompletion.create函数的调用不再需要prompt参数，而是换成了messages参数，并且，不同于prompt参数对象是以简单的字符串形式呈现，messages参数则是一个基本构成元素为字典的列表，其内每个字典都代表一条独立的消息，每个字典都包含两个键值（Key-value）对，其中第一个Key都是字符串role（角色）表示某条消息的作者，第二个key为content（内容）表示消息具体内容。可以说messages参数是ChatCompletion.create函数最重要的参数之一，能够看出比简单的prompt参数格式要更加复杂。更多关于message的参数设置方法稍后介绍，总的来看，这里的messages就可以简单理解为输入给模型的信息，而模型接收到message之后也会输出对应的回答信息，当然也是以message形式呈现：

In [18]:
# 返回对象类型和Completion.create函数返回对象类型一致
type(response)

openai.openai_object.OpenAIObject

In [19]:
# 可以通过索引的方式索引出文本部分内容
response["choices"]

[<OpenAIObject at 0x2017fa89ea0> JSON: {
   "finish_reason": "stop",
   "index": 0,
   "message": {
     "content": "\u673a\u5668\u5b66\u4e60\u662f\u4e00\u79cd\u4eba\u5de5\u667a\u80fd\u7684\u5206\u652f\u9886\u57df\uff0c\u5b83\u7814\u7a76\u5982\u4f55\u4f7f\u8ba1\u7b97\u673a\u7cfb\u7edf\u6839\u636e\u6570\u636e\u548c\u7ecf\u9a8c\u81ea\u52a8\u6539\u5584\u548c\u9002\u5e94\uff0c\u800c\u65e0\u9700\u660e\u786e\u5730\u8fdb\u884c\u7f16\u7a0b\u3002\u673a\u5668\u5b66\u4e60\u7684\u76ee\u6807\u662f\u901a\u8fc7\u6784\u5efa\u6a21\u578b\u548c\u7b97\u6cd5\uff0c\u8ba9\u8ba1\u7b97\u673a\u80fd\u591f\u4ece\u5927\u91cf\u6570\u636e\u4e2d\u5b66\u4e60\uff0c\u5e76\u4e14\u53ef\u4ee5\u4ece\u4e2d\u53d1\u73b0\u6a21\u5f0f\u3001\u505a\u51fa\u9884\u6d4b\u548c\u505a\u51fa\u76f8\u5e94\u7684\u51b3\u7b56\u3002\u673a\u5668\u5b66\u4e60\u6280\u672f\u88ab\u5e7f\u6cdb\u5e94\u7528\u4e8e\u56fe\u50cf\u548c\u8bed\u97f3\u8bc6\u522b\u3001\u81ea\u7136\u8bed\u8a00\u5904\u7406\u3001\u63a8\u8350\u7cfb\u7edf\u3001\u6570\u636e\u6316\u6398\

In [20]:
# 也可以通过属性调用的方式查看choices
response.choices

[<OpenAIObject at 0x2017fa89ea0> JSON: {
   "finish_reason": "stop",
   "index": 0,
   "message": {
     "content": "\u673a\u5668\u5b66\u4e60\u662f\u4e00\u79cd\u4eba\u5de5\u667a\u80fd\u7684\u5206\u652f\u9886\u57df\uff0c\u5b83\u7814\u7a76\u5982\u4f55\u4f7f\u8ba1\u7b97\u673a\u7cfb\u7edf\u6839\u636e\u6570\u636e\u548c\u7ecf\u9a8c\u81ea\u52a8\u6539\u5584\u548c\u9002\u5e94\uff0c\u800c\u65e0\u9700\u660e\u786e\u5730\u8fdb\u884c\u7f16\u7a0b\u3002\u673a\u5668\u5b66\u4e60\u7684\u76ee\u6807\u662f\u901a\u8fc7\u6784\u5efa\u6a21\u578b\u548c\u7b97\u6cd5\uff0c\u8ba9\u8ba1\u7b97\u673a\u80fd\u591f\u4ece\u5927\u91cf\u6570\u636e\u4e2d\u5b66\u4e60\uff0c\u5e76\u4e14\u53ef\u4ee5\u4ece\u4e2d\u53d1\u73b0\u6a21\u5f0f\u3001\u505a\u51fa\u9884\u6d4b\u548c\u505a\u51fa\u76f8\u5e94\u7684\u51b3\u7b56\u3002\u673a\u5668\u5b66\u4e60\u6280\u672f\u88ab\u5e7f\u6cdb\u5e94\u7528\u4e8e\u56fe\u50cf\u548c\u8bed\u97f3\u8bc6\u522b\u3001\u81ea\u7136\u8bed\u8a00\u5904\u7406\u3001\u63a8\u8350\u7cfb\u7edf\u3001\u6570\u636e\u6316\u6398\

In [21]:
# 查看第一个返回结果
response.choices[0]

<OpenAIObject at 0x2017fa89ea0> JSON: {
  "finish_reason": "stop",
  "index": 0,
  "message": {
    "content": "\u673a\u5668\u5b66\u4e60\u662f\u4e00\u79cd\u4eba\u5de5\u667a\u80fd\u7684\u5206\u652f\u9886\u57df\uff0c\u5b83\u7814\u7a76\u5982\u4f55\u4f7f\u8ba1\u7b97\u673a\u7cfb\u7edf\u6839\u636e\u6570\u636e\u548c\u7ecf\u9a8c\u81ea\u52a8\u6539\u5584\u548c\u9002\u5e94\uff0c\u800c\u65e0\u9700\u660e\u786e\u5730\u8fdb\u884c\u7f16\u7a0b\u3002\u673a\u5668\u5b66\u4e60\u7684\u76ee\u6807\u662f\u901a\u8fc7\u6784\u5efa\u6a21\u578b\u548c\u7b97\u6cd5\uff0c\u8ba9\u8ba1\u7b97\u673a\u80fd\u591f\u4ece\u5927\u91cf\u6570\u636e\u4e2d\u5b66\u4e60\uff0c\u5e76\u4e14\u53ef\u4ee5\u4ece\u4e2d\u53d1\u73b0\u6a21\u5f0f\u3001\u505a\u51fa\u9884\u6d4b\u548c\u505a\u51fa\u76f8\u5e94\u7684\u51b3\u7b56\u3002\u673a\u5668\u5b66\u4e60\u6280\u672f\u88ab\u5e7f\u6cdb\u5e94\u7528\u4e8e\u56fe\u50cf\u548c\u8bed\u97f3\u8bc6\u522b\u3001\u81ea\u7136\u8bed\u8a00\u5904\u7406\u3001\u63a8\u8350\u7cfb\u7edf\u3001\u6570\u636e\u6316\u6398\u7b49

In [23]:
# 查看第一个返回结果的message
response.choices[0].message['content']

'机器学习是一种人工智能的分支领域，它研究如何使计算机系统根据数据和经验自动改善和适应，而无需明确地进行编程。机器学习的目标是通过构建模型和算法，让计算机能够从大量数据中学习，并且可以从中发现模式、做出预测和做出相应的决策。机器学习技术被广泛应用于图像和语音识别、自然语言处理、推荐系统、数据挖掘等领域，以解决复杂问题和改进人工智能系统的性能。'

可以说，在获取对话结果的过程中，除了ChatCompletion.create返回结果最终是保存在message属性中（Completion.create返回结果是保存在text属性中），其他方面ChatCompletion.create的返回结果和Completion.create返回结果完全一致。而文本补全模型需要输入“prompt”，输出结果是“text”；而对话模型需要输入“message”，返回也是“message”，可以说这种文本交互形式确实非常符合人类在进行聊天时的问答习惯。

&emsp;&emsp;最后需要注意的是，和Completion模型一样，Chat模型我们同样也可以在返回结果的usage中查看本次对话所占用的token数量，其中"prompt_tokens"表示提示词占用token数量，"completion_tokens"则表示返回结果所占用token数量，而"total_tokens"则是二者相加，代表本次对话总共占用token数量。

In [46]:
response.usage

<OpenAIObject at 0x2e70131fb50> JSON: {
  "prompt_tokens": 20,
  "completion_tokens": 464,
  "total_tokens": 484
}

In [47]:
response.usage["total_tokens"]

484

#### 2.2 ChatCompletion.create函数参数解释

&emsp;&emsp;而关于ChatCompletion.create函数的详细参数解释，可以在官网相关页面查阅：https://platform.openai.com/docs/api-reference/chat/create 。能够发现，和Completion.create函数相比，ChatCompletion.create函数的参数结构发生了以下变化：     
- 用messages参数代替了prompt参数，使之更适合能够执行对话类任务；
- 新增functions和function_call参数，使之能够在函数内部调用其他工具的API，该功能为0613新增功能；
- 其他核心参数完全一致，例如temperature、top_p、max_tokens、n、presence_penalty等参数的解释和使用方法都完全一致，且这些参数具体的调整策略也完全一致，例如如果希望具备更有创造力的对话，则可以调高temperature且降低presence_penalty，而如果希望获得更加严谨的问答结果，则可以降低temperature并且提高presence_penalty取值；
- 剔除了best_of参数，即Chat模型不再支持从多个答案中选择一个最好的答案这一功能；

需要注意的是，两个函数相同的参数在Ch.2的Completion.create函数参数介绍中均有详细介绍，此处不再赘述。接下来我们围绕ChatCompletion.create不同的三个参数，即messages参数、functions和function_call参数进行详细解释。

### 3.messages参数详解

#### 3.1 message参数结构及功能解释

&emsp;&emsp;接下来我们来看ChatCompletion.create函数非常重要的参数——messages使用方法。总的来说，messages是一种用于描述ChatCompletion模型和用户之间通信信息的高级抽象，从表示形式上来说，一个messages是一个列表，包含多个字典，而每个字典都是一条消息，其中，一条消息由包含两个键值对（即每个字典都包含两个键值对），第一个键值对用于表示消息发送者，其中第一个Key为字符串'role'，Value为参与对话的角色名称，或者可以理解为本条消息的作者或消息发送人名称，第二个键值对表示具体消息内容，Key为字符串'content'，Value为具体的消息内容，用字符串表示。

> 实际上，根据官网的说明，更严谨的说法是role是content的作者，而content的作者并不一定是content的发送方，发送方的角色更多是用于消息传递而非消息创作。但在实际使用过程中我们发现，ChatCompletion模型的role几乎可以完全看成是消息发送方，这么理解也会更加便于我们对消息结构的掌握和解读。

例如上述示例中的messages就总共包含一条信息，即一个一个名为user的角色发送了一条名为'请问什么是机器学习？'的消息：

In [26]:
messages=[
    {"role": "user", "content": "请问什么是机器学习？"}
]

而同时，返回的message结果也是一个“字典”，并且也包含了信息的发送方和具体信息内容，不难看出，此时返回的message发送方是一个名为'assistant'的角色，而具体内容则是一段关于什么是机器学习的描述：

In [27]:
response.choices[0].message

<OpenAIObject at 0x2017fa89d10> JSON: {
  "content": "\u673a\u5668\u5b66\u4e60\u662f\u4e00\u79cd\u4eba\u5de5\u667a\u80fd\u7684\u5206\u652f\u9886\u57df\uff0c\u5b83\u7814\u7a76\u5982\u4f55\u4f7f\u8ba1\u7b97\u673a\u7cfb\u7edf\u6839\u636e\u6570\u636e\u548c\u7ecf\u9a8c\u81ea\u52a8\u6539\u5584\u548c\u9002\u5e94\uff0c\u800c\u65e0\u9700\u660e\u786e\u5730\u8fdb\u884c\u7f16\u7a0b\u3002\u673a\u5668\u5b66\u4e60\u7684\u76ee\u6807\u662f\u901a\u8fc7\u6784\u5efa\u6a21\u578b\u548c\u7b97\u6cd5\uff0c\u8ba9\u8ba1\u7b97\u673a\u80fd\u591f\u4ece\u5927\u91cf\u6570\u636e\u4e2d\u5b66\u4e60\uff0c\u5e76\u4e14\u53ef\u4ee5\u4ece\u4e2d\u53d1\u73b0\u6a21\u5f0f\u3001\u505a\u51fa\u9884\u6d4b\u548c\u505a\u51fa\u76f8\u5e94\u7684\u51b3\u7b56\u3002\u673a\u5668\u5b66\u4e60\u6280\u672f\u88ab\u5e7f\u6cdb\u5e94\u7528\u4e8e\u56fe\u50cf\u548c\u8bed\u97f3\u8bc6\u522b\u3001\u81ea\u7136\u8bed\u8a00\u5904\u7406\u3001\u63a8\u8350\u7cfb\u7edf\u3001\u6570\u636e\u6316\u6398\u7b49\u9886\u57df\uff0c\u4ee5\u89e3\u51b3\u590d\u6742\u95ee\u98

In [28]:
response.choices[0].message['content']

'机器学习是一种人工智能的分支领域，它研究如何使计算机系统根据数据和经验自动改善和适应，而无需明确地进行编程。机器学习的目标是通过构建模型和算法，让计算机能够从大量数据中学习，并且可以从中发现模式、做出预测和做出相应的决策。机器学习技术被广泛应用于图像和语音识别、自然语言处理、推荐系统、数据挖掘等领域，以解决复杂问题和改进人工智能系统的性能。'

由此不难看出，对话Chat模型的每个对话任务都是通过输入和输出message来完成的。

#### 3.2 messages中的角色划分

- user role和assistant role

&emsp;&emsp;那么接下来的问题就是，在实际调用Chat模型进行对话时，messages中的role应该如何设置呢？从上述极简的对话示例中能够看出，一个最简单的对话就是我们扮演user（用户）这个角色（'role':'user'），然后在content中输入我们的问题并等待模型回答。而模型在实际回答过程中，也会扮演一个名为assistant（助手）这个角色（'role':'assistant'）进行回答，这里的user和assistant是具有明确含义的字符串，即如果一条信息的role是user，则表明这是用户向模型发送的聊天信息，相当于是Completion模型中的prompt，而如果一条信息的role是assistant，则表示这是当前模型围绕某条用户信息做出的回应，相当于是相当于是Completion模型中的text。需要注意的是，在messages参数中，我们是不能给自己或者模型自定义其他名称的。

&emsp;&emsp;很明显，基于这样的一个定义的规则，最简单的Chat模型的调用方法就是在messages参数中设置一条role为user的参数，在content中输入聊天的内容，而模型则会根据这条用户输入给模型的消息进行回答，类似于此前我们向模型提问“请问什么是机器学习？”这种提问方式:

In [29]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "请问什么是机器学习？"}
  ]
)

In [30]:
response.choices[0].message['content']

'机器学习是一种人工智能的分支，通过使用数学和统计方法来自动分析和理解数据，并使计算机能够自主学习和改进性能。它是通过从数据中学习模式和规律，而不是通过明确编程来实现任务的一种方法。机器学习可以帮助计算机系统通过从大量数据中提取规律来预测结果、进行分类、聚类、识别图像和语音、自动驾驶等。'

不过需要注意的是，尽管一个messages可以包含多条信息，但模型只会对于最后一条用户信息进行回答，例如：

In [37]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "请问什么是机器学习？"},
    {"role": "user", "content": "请问什么是决策树算法？"}
  ]
)

In [38]:
response.choices[0].message['content']

'决策树算法是一种用于分类和回归的机器学习算法，它基于树形结构来进行决策。决策树可以看作是一个由节点和有向边组成的树，每个节点表示一个特征属性，每条边表示一个特征属性取值或决策的结果。决策树的根节点表示最重要的特征，而叶子节点表示分类或回归结果。\n\n决策树算法的特点包括易于理解和解释、能够处理非线性关系、不需要对数据进行特征预处理、可以处理多输出问题等。算法的核心思想是通过反复分割数据集，使得每个子集中的样本都属于同一类别（或拥有相似的回归结果）。分割的决策依据是选择最佳的划分特征，通常使用各种不纯度指标（例如基尼系数或熵）来衡量每个特征的纯度。\n\n决策树算法包括很多变种和优化方法，如CART（Classification and Regression Trees）、ID3、C4.5、随机森林等。决策树算法在实际应用中广泛使用，可以用于分类问题（如垃圾邮件分类、医学诊断等）和回归问题（如房价预测、股票趋势预测等）。'

也就是说，assistant消息和role消息是一一对应的，而且在一般情况下，assistant消息只会围绕messages参数中的最后一个role信息进行回答。

- system role用于身份设定

&emsp;&emsp;不过，值得一提的是，user和assistant的这种提问方式尽管足够清晰，但往往形式上不够丰富，例如在实践中人们发现，给聊天机器人进行一个身份设置，其实是非常有效的引导模型创作我们想要的结果的方法，例如如果我们希望获得一个关于“什么是机器学习？”更加严谨且丰富的答案，我们可以以“假设你是一名资深的计算机系大学教授”为模型进行身份设置，例如我们可以以如下方式向模型进行提问：

In [33]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "假设你是一名资深的计算机系大学教授，请帮我回答，什么是机器学习？"}
  ]
)

In [34]:
response.choices[0].message['content']

'机器学习是一门研究如何使计算机系统通过大量数据和经验自动学习和改进性能的领域。它利用算法和数学模型，使计算机能够自动发现数据中的模式和规律，并利用这些模式来进行预测、分类、优化等任务。\n\n在机器学习中，计算机系统并不需要明确地编写具体规则或指令，而是通过从数据中学习，从而提高自己的性能和准确性。它依赖于大数据、统计学和计算机科学等领域的技术，包括数据预处理、特征提取、模型选择和评估等步骤。\n\n机器学习广泛应用于各个领域，如自然语言处理、计算机视觉、推荐系统、金融预测等。它的目标是从数据中发现隐藏的模式和知识，以提供高效的决策支持和智能化的解决方案。'

不难看出，此时模型的回答就变得更加详细和严谨，更像一名“大学教授”的语气风格，也同时说明我们对模型进行的身份设定是切实有效的。

&emsp;&emsp;而在ChatCompletion.create函数中，还有另外一种非常便捷的对模型进行身份设置的方法，即使用system role，即我们可以使用如下方式为模型进行“大学教授”身份设定：

In [41]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "你是一名资深的计算机系大学教授"},
    {"role": "user", "content": "请问什么是机器学习？"}
  ]
)

In [42]:
response.choices[0].message['content']

'机器学习是一种人工智能的分支领域，它致力于研究和开发能够从数据中自动学习和改进的算法和模型。机器学习使计算机可以通过分析大量数据并自动发现数据中隐藏的模式和规律，从而使计算机能够作出预测或做出决策。\n\n机器学习的主要任务包括分类、回归、聚类和推荐等。分类任务是将数据分为不同的类别，回归任务是预测数据的数值，聚类任务是将相似的数据分组，推荐任务是根据用户的个人偏好为其推荐物品。\n\n机器学习算法可以分为监督学习、无监督学习和强化学习。监督学习是通过已标记的训练数据进行学习，无监督学习是利用未标记的数据进行学习，而强化学习则是通过与环境的交互来进行学习。\n\n机器学习在许多领域中都有广泛的应用，例如图像识别、语音识别、自然语言处理、金融预测和医学诊断等。通过机器学习，计算机可以从大量的数据中学习，并提供更准确、高效的解决方案。'

&emsp;&emsp;能够看出，这里我们在原有消息之前，新增一条消息{"role": "system", "content": "你是一名资深的计算机系大学教授"}，也能起到设定模型身份的作用。而这条消息的实际含义是，以system的身份发送一条消息，消息内容为“你是一名资深的计算机系大学教授”。这里的system就是messages参数的role可以选取的第三个字符串，意为该消息为一条系统消息。相比用户消息，系统消息有以下几点需要注意，其一是系统消息的实际作用是给整个对话系统进行背景设置，不同的背景设置会极大程度影响后续对话过程中模型的输出结果，例如如果系统设置为“你是一位资深医学专家”，那么接下来系统在进行回答医学领域相关问题时则会引用大量医学术语，而如果系统设置为“你是一位资深喜剧演员”，那么接下来系统进行的回答则会更加风趣幽默：

In [45]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "你是一名资深的喜剧演员"},
    {"role": "user", "content": "请问什么是机器学习？"}
  ]
)

In [46]:
response.choices[0].message['content']

'哈哈，这是一个有趣的问题！机器学习是一种人工智能的分支，它通过让计算机系统自动学习和改进，而无需明确的编程指令。简单来说，机器学习是让机器具备从数据中学习和预测的能力。就像我在喜剧表演中学习和改进我的技巧一样，机器学习通过分析大量的数据，找到模式和规律，从而使机器能够做出智能的决策和预测。不过相比我，机器学习更加准确和高效！话不多说，让我们继续我的喜剧表演吧！'

&emsp;&emsp;而第二方面需要注意的则是，当messages中只包含一条system消息时，系统会围绕system进行回答，只不过此时系统的assistant的应答消息则更像是一个completion的过程，即围绕system的prompt进行进一步的文本补全：

In [47]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "你是一名资深的喜剧演员"},
  ]
)

In [49]:
response.choices[0].message

<OpenAIObject at 0x2017fa96590> JSON: {
  "content": "\u6211\u4f1a\u7ed9\u89c2\u4f17\u5e26\u6765\u6b22\u4e50\u548c\u5feb\u4e50\u7684\u7b11\u58f0\uff01\u6211\u64c5\u957f\u5851\u9020\u4e30\u5bcc\u591a\u6837\u7684\u89d2\u8272\uff0c\u8fd0\u7528\u5938\u5f20\u7684\u8868\u6f14\u6280\u5de7\u548c\u5e7d\u9ed8\u7684\u53f0\u8bcd\uff0c\u8ba9\u89c2\u4f17\u6367\u8179\u5927\u7b11\u3002\u6211\u559c\u6b22\u6311\u6218\u81ea\u5df1\uff0c\u5c1d\u8bd5\u4e0d\u540c\u7684\u559c\u5267\u98ce\u683c\uff0c\u4f8b\u5982\u60c5\u666f\u559c\u5267\u3001\u559c\u5267\u7535\u5f71\u3001Stand-up\u559c\u5267\u7b49\u7b49\u3002\u901a\u8fc7\u4ee4\u4eba\u96be\u5fd8\u7684\u8868\u6f14\uff0c\u6211\u5e0c\u671b\u80fd\u591f\u7559\u4e0b\u6df1\u523b\u7684\u5370\u8c61\uff0c\u5e76\u8ba9\u89c2\u4f17\u5fd8\u8bb0\u751f\u6d3b\u4e2d\u7684\u70e6\u607c\uff0c\u4eab\u53d7\u4e00\u6bb5\u6b22\u4e50\u7684\u65f6\u5149\uff01",
  "role": "assistant"
}

In [48]:
response.choices[0].message['content']

'我会给观众带来欢乐和快乐的笑声！我擅长塑造丰富多样的角色，运用夸张的表演技巧和幽默的台词，让观众捧腹大笑。我喜欢挑战自己，尝试不同的喜剧风格，例如情景喜剧、喜剧电影、Stand-up喜剧等等。通过令人难忘的表演，我希望能够留下深刻的印象，并让观众忘记生活中的烦恼，享受一段欢乐的时光！'

In [53]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "你好，请问"},
  ]
)

In [54]:
response.choices[0].message['content']

'有什么可以帮助您的吗？'

当然，借助completion过程也是可以进行提问的，例如：

In [50]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "请问什么是机器学习？"},
  ]
)

In [52]:
response.choices[0].message['content']

'机器学习是一种人工智能的分支，它通过使用计算机算法和模型，让机器自动从数据中学习，并从中提取规律和模式，以做出预测和决策。它的主要目标是使机器能够从经验中学习，提高预测能力和自动化任务。机器学习在许多领域都有广泛应用，包括自然语言处理、图像识别、数据挖掘和智能推荐系统等。'

不过这么做意义并不大，还是建议以user角色进行提问。

&emsp;&emsp;第三方面需要注意的是，如果我们需要根据system系统信息对系统进行设置，然后再提问，那么先system消息再user消息的顺序就变得非常重要，例如还是上面的例子，还是希望以喜剧演员的身份介绍机器学习，但我们调换了system消息和user消息的顺序，那么会发现，system消息的作用就会失效：

In [55]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "请问什么是机器学习？"},
    {"role": "system", "content": "你是一名资深的喜剧演员"}
  ]
)

In [56]:
response.choices[0].message['content']

'机器学习是一门人工智能领域的研究，旨在让计算机系统通过学习和改进经验，不需要明确的程序指导下，自动完成特定任务。它利用统计学和算法来使计算机能够从数据中学习并进行预测和决策。机器学习可以应用于各种领域，如图像识别、语音识别、自然语言处理、推荐系统等。通过训练模型，机器学习可以帮助计算机系统能够从数据中识别模式，做出预测，并根据反馈不断改进自己的表现。'

此时会发现，模型还是能解答“请问什么是机器学习？”这个问题，但却没有正确接受“你是一名资深喜剧演员”这个设定。

&emsp;&emsp;最后还有一点需要注意的是，根据OpenAI官网说明，截至目前，gpt-3.5系列模型仍然无法对system提供的系统消息保持长期关注，即在多轮对话中，模型极有可能逐渐忘记自己的身份设定。根据长期使用情况来看，gpt-4模型对system设置的长期关注要好于gpt-3.5系列模型。

&emsp;&emsp;对于messages中可选的role来说，除了ueser、assistant、system之外，还有一个function role，用于表示某条消息为某函数的调用指令，function role是OpenAI 0613更新中提供的新的role选项，用于Chat模型调用外部定义函数或者工具API时使用。相关用法我们会在本小节的后半部分进行介绍。

### 4.messages参数应用

&emsp;&emsp;对于ChatCompletion.create函数来说，通过灵活的messages参数，能够非常便捷高效的实现诸多类型的对话需求，例如基于提示词模板的提问、Few-shot提问、基于某背景知识的提问等，接下来我们进一步介绍不同场景下messages参数的设置方法，并在这个过程进一步介绍关于messages中name的设置方法。

&emsp;&emsp;由于我们接下来需要使用此前Ch.4中的推理问题作为示例，因此我们提前定义好四组问题的问题和答案：

In [68]:
Q1 = '罗杰有五个网球，他又买了两盒网球，每盒有3个网球，请问他现在总共有多少个网球？'
A1 = '现在罗杰总共有11个网球。'

In [69]:
Q2 = '食堂总共有23个苹果，如果他们用掉20个苹果，然后又买了6个苹果，请问现在食堂总共有多少个苹果？'
A2 = '现在食堂总共有9个苹果。'

In [70]:
Q3 = '杂耍者可以杂耍16个球。其中一半的球是高尔夫球，其中一半的高尔夫球是蓝色的。请问总共有多少个蓝色高尔夫球？'
A3 = '现在总共有4个蓝色高尔夫球。'

In [71]:
Q4 = '艾米需要4分钟才能爬到滑梯顶部，她花了1分钟才滑下来，水滑梯将在15分钟后关闭，请问在关闭之前她能滑多少次？'
A4 = '关闭之前艾米能滑3次。'

> 这里需要注意的是，gpt-3.5在解决推理问题上同样展示出了涌现能力——即gpt-3.5是基于对话语料进行的微调，但却展示出了比text-davinci-003更强大的推理能力，此前的四个推理问题对于gpt-3.5来说，除了最后一个问题不一定能得出正确答案外，其他问题均能在不进行额外提示的情况下进行很好的回答。而经过测试，gpt-4模型能够非常好的回答第四个推理问题。但需要注意的是，这并不代表此前的CoT和LtM技术就不再重要，面对超出模型原生能力的更加复杂的推理问题（如SCAN数据集的命令解释问题），仍然还是需要使用这些提示工程技术。当然，本节的重点是介绍messages参数设置技巧，更多的关于Chat模型的实战应用详见下一小节。

- 借助多轮user-assistant消息进行few-shot

&emsp;&emsp;如果想在Chat模型中进行Few-shot，最好的办法就是在messages中设置多轮user-assistant消息，这里我们还是以Ch.4中的推理问题为例，尝试在Chat模型中进行推理并进行Few-shot，这里我们尝试以第一个问题的问题和答案作为提示示例，引导模型解答第二个问题，则可以按照如下方式设置messages：

In [112]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": Q1},
    {"role": "assistant", "content": A1},
    {"role": "user", "content": Q2}
  ]
)

In [113]:
response.choices[0].message['content']

'现在食堂总共有9个苹果。'

而相比单独进行Q2的提问，经过Few-shot的提示回答的结果，会更加接近A1结果的表示格式：

In [110]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": Q2}
  ]
)

In [111]:
response.choices[0].message['content']

'食堂原本有23个苹果，用掉了20个，剩下3个苹果。\n然后又买了6个苹果，所以现在食堂总共有3 + 6 = 9个苹果。'

In [114]:
A1

'现在罗杰总共有11个网球。'

&emsp;&emsp;在这个示例中，我们能够看出其实assistant消息也是可以自定义的，用于给模型提供回答的范本。并且由此可见messages参数具体呈现形式可以非常多样，不仅可以按照system-user的形式规定回答风格，而且还可以按照user-assistant-user-assistant...形式来进行Few-shot。这其实也是得益于messages高度灵活的参数构成形式——即由多条注明消息源的消息构成。messages中具体每条消息的内容以及消息序列构成形式上，有非常大的调整的空间。

- 借助system role进行Few-shot

&emsp;&emsp;当然，除了可以借助多轮user和assistant消息来进行Few-shot外，我们还可以把提示示例写进一条system信息中，作为当前问答的背景信息，例如：

In [122]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": 'Q: ' + Q1 + 'A: ' + A1},
    {"role": "user", "content": 'Q: ' + Q2 }
  ]
)

In [123]:
response.choices[0].message['content']

'A: 现在食堂总共有9个苹果。'

In [120]:
'Q: ' + Q1 + 'A: ' + A1

'Q: 罗杰有五个网球，他又买了两盒网球，每盒有3个网球，请问他现在总共有多少个网球？A: 现在罗杰总共有11个网球。'

In [121]:
'Q: ' + Q2 

'Q: 食堂总共有23个苹果，如果他们用掉20个苹果，然后又买了6个苹果，请问现在食堂总共有多少个苹果？'

唯一需要注意的是，此时需要在系统消息中通过Q和A的方式注明问题和回答的内容，并且在提问时添加字符串Q，便于模型理解后续的回答需要按照system中的A的方式进行回答。

&emsp;&emsp;当然，这种system的设置方式还是会非常类似于prompt编写方式。那么，有没有更加简单的、借助system消息、不用单独设置Q和A的方式来完成Few-shot呢？这里有一种更为简单的方法，即设置两条system消息，此时模型会默认这两条system消息就是一次user-assistant对话消息，即可以按照如下方式完成Few-shot：

In [126]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": Q1},
    {"role": "system", "content": A1},
    {"role": "user", "content": Q2}
  ]
)

In [127]:
response.choices[0].message['content']

'现在食堂总共有9个苹果。'

不过这种代码形式可读性并不强，如希望通过多条system消息来模拟user-assistant多轮对话消息，则最好在system消息中添加一组额外的Key-value，相当于给这条消息额外进行备注，其中Key需要输入字符串name，而value则需要输入有助于理解这条消息定位的备注信息，例如：

In [124]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "name":"example_user", "content": Q1},
    {"role": "system", "name":"example_assistant", "content": A1},
    {"role": "user", "content": Q2}
  ]
)

In [125]:
response.choices[0].message['content']

'现在食堂总共有9个苹果。'

其中，前两条system消息分别输入了"name":"example_user"和"name":"example_assistant"，表示这两条消息中第一条消息用于模拟用户消息，第二条消息用于模拟助手消息。不过需要注意的是，当role为user、assistant或system时，messages参数中的字典的name参数是可选参数，且并无实际作用，并不会对模型的输出结果造成影响，name只用于标识特定的消息或对话轮次，相当于是增强代码可读性的。在很多多轮对话场景中，我们可以通过设置name参数来更好地组织和跟踪对话的不同部分。而需要注意的是，若role为function，则name参数就是必选参数，相关内容我们会在介绍function role时进行介绍。

- 借助system role输入提示模板

&emsp;&emsp;即然system消息能够作为背景设定的基本消息并对后续的问答消息造成影响，那么很容易想到的system消息的一个应用场景就是借助system role输入提示模板，例如在Ch.4中曾介绍到，为了更好的提高模型推理能力，我们可以在每个prompt中加入一句“请一步步推理并得出结论”进而实现Zero-shot-CoT。而在Chat模型中，这种prompt模板信息是非常适合通过system role进行输入的，例如围绕第四个推理问题，我们可以通过输入一条内容为“请一步步推理并得出结论”的系统信息，来引导模型完成Zero-shot-CoT：

In [131]:
prompt_temp_cot = '请一步步思考并解决问题'

In [132]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": prompt_temp_cot},
    {"role": "user", "content": Q1}
  ]
)

In [133]:
response.choices[0].message['content']

'罗杰原本有5个网球\n他又买了两盒网球，每盒有3个，所以一共买了2盒 × 3个/盒 = 6个网球\n总共的网球数量为5个 + 6个 = 11个网球'

能够看出，模型确实开始进行了CoT推导。接下来我们测试第四个推理问题、也是最复杂的一个推理问题：

In [59]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": prompt_temp_cot},
    {"role": "user", "content": Q4}
  ]
)

In [60]:
response.choices[0].message['content']

'首先，我们需要弄清楚在15分钟内，艾米能够滑下来多少次。\n\n艾米每次从滑梯顶部滑下来需要1分钟，所以在15分钟内，她最多可以滑下来15次。\n\n然后，我们需要确定艾米每次滑下来后需要多少时间才能再次爬到滑梯顶部。\n\n艾米每次滑下来耗时1分钟，所以她需要再花4分钟才能爬到滑梯顶部。所以总共耗时为1分钟下滑时间加上4分钟爬升时间，即为5分钟。\n\n最后，我们可以计算出在15分钟内，艾米可以滑多少次。\n\n15分钟总共有多少个5分钟？答案是15/5=3个。\n\n艾米每次滑下来需要1分钟，所以在15分钟内，她可以滑3次。\n\n因此，在关闭之前，艾米可以滑3次。'

能够看出发现在Zero-shot-CoT的情况下，gpt-3.5模型能够推导得出第四个问题的正确答案，但过程有点绕。

&emsp;&emsp;接下来我们测试LtM提示法：

In [145]:
prompt_temp_ltm = '为了解决当前这个问题，请列举我们先要解决的问题，并逐步解决原问题。'

In [136]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": prompt_temp_ltm},
    {"role": "user", "content": Q1}
  ]
)

In [137]:
response.choices[0].message['content']

'先要解决的问题：\n1. 罗杰原来有五个网球，他买了两盒网球，每盒有三个网球，我们需要计算他购买的网球数量。\n\n逐步解决：\n1. 罗杰原来有五个网球。\n2. 他买了两盒网球，每盒有三个网球。两盒网球总共有 2 * 3 = 6 个网球。\n3. 罗杰原来有的五个网球加上购买的六个网球，总共有 5 + 6 = 11 个网球。 \n\n所以罗杰现在总共有 11 个网球。'

能够发现模型确实能够按照提示模板进行逐步问题解决，并且和text-davinci-003模型类似，都能够一次性解决前置问题以及原问题。

&emsp;&emsp;接下来继续测试第四个问题：

In [150]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": prompt_temp_ltm},
    {"role": "user", "content": Q4}
  ]
)

In [151]:
response.choices[0].message['content']

'要解决这个问题，我们需要考虑以下几个问题并逐步解决：\n\n1. 每次滑下来的时间是多少？\n2. 在15分钟之内能滑下来多少次？\n3. 是否需要考虑滑下来后到达滑梯顶部需要的时间？\n\n解决上述问题后，我们可以得到在关闭之前艾米能滑的次数。现在我们逐个解决这些问题：\n\n1. 每次滑下来的时间是多少？\n已知艾米花了1分钟才滑下来，所以每次滑下来的时间是1分钟。\n\n2. 在15分钟之内能滑下来多少次？\n在15分钟之内，每次滑下来的时间是1分钟，所以艾米在15分钟之内能滑的次数是15/1=15次。\n\n3. 是否需要考虑滑下来后到达滑梯顶部需要的时间？\n根据题目描述，我们知道艾米需要4分钟才能爬到滑梯顶部。如果考虑滑下来后再爬上去的时间，每次滑梯的周期是5分钟（1分钟滑下来+4分钟爬上去）。\n\n所以，在15分钟之内，艾米能滑的次数是15/5=3次。\n\n所以，在关闭之前艾米能滑3次。'

能够发现，ltm提示方法对gpt-3.5这类Chat模型仍然有效。

---

- gpt-4性能测试

&emsp;&emsp;相比gpt-3.5复杂且不稳定的推导过程，gpt-4模型在相同提示策略下，会表现出更强大的推理性能。例如，同样是围绕第四个问题，对gpt-4模型进行CoT和LtM提示结果如下：

In [61]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[
    {"role": "system", "content": prompt_temp_cot},
    {"role": "user", "content": Q4}
  ]
)

In [62]:
response.choices[0].message['content']

'第一步，首先需要确定艾米完成一个滑梯运动需要的总时间，即爬到滑梯顶部的时间与滑下来的时间总和。在此问题中，艾米需要4分钟爬到滑梯顶部，1分钟滑下来。总共需要5分钟。\n\n第二步，然后我们需要计算在水滑梯关闭前，艾米可以完成多少次滑梯运动。我们已知水滑梯将在15分钟后关闭，因此我们将15分钟除以每次滑梯运动所需要的时间，得到她可以滑的次数。\n\n计算为：15分钟 ÷ 5分钟/次 = 3次\n\n所以，在水滑梯关闭之前，艾米可以滑3次。'

In [152]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[
    {"role": "system", "content": prompt_temp_ltm},
    {"role": "user", "content": Q4}
  ]
)

In [153]:
response.choices[0].message['content']

'要解决这个问题，我们需要首先解决以下几个子问题：\n\n1. 确定艾米每次滑梯需要的总时间，即爬到滑梯顶部和滑下来的时间总和。\n2. 确定在水滑梯关闭前，艾米有多少时间可以滑梯。\n3. 使用总时间除以每次滑梯所需的时间。\n\n解决方案如下：\n\n1. 每次滑梯艾米需要的总时间是4分钟+1分钟=5分钟。\n2. 水滑梯将在15分钟后关闭，所以艾米有15分钟的时间可以滑梯。\n3. 艾米能滑的次数=总时间/每次滑梯所需的时间=15分钟/5分钟=3次。\n\n所以，艾米在水滑梯关闭前能滑3次。'

能够看出，gpt-4模型展示出的推理性能要强大很多。当然，除了推理能力外，gpt-4各方面性能都属于目前大模型领域的顶流。

---

- 借助system role设置聊天背景信息

&emsp;&emsp;除了可以借助system消息非常便捷的进行提示模板的设计之外，还有一个非常常见的system role的使用方法，就是借助system消息进行聊天背景信息的设定，很多时候我们可以在system消息中输入一段长文本，这段长文本将在聊天开始之前输入到系统中，而在之后的聊天中，即可让assistant围绕这个长文本进行回答，这是一种最简单的实现大语言模型围绕本地知识库进行问答的方法。

&emsp;&emsp;这里我们在system消息中输入一段关于虚拟人物“陈明”的个人简介，而在之后的提问中，user和assistant将可以自由的围绕这段输入的背景信息进行问答：

In [3]:
text = '陈明，男，1973年7月15日出生于中国福建省厦门市。\
        1991年毕业于厦门大学电子科学与技术系，继而于1998年在北京大学获得信息技术博士学位。\
        毕业后的陈明在硅谷的一家著名科技公司工作了五年，专注于人工智能和机器学习的研发。'

In [4]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": text},
    {"role": "user", "content": '请问陈明是那一年出生？'}
  ]
)

In [5]:
response.choices[0].message['content']

'陈明于1973年出生。'

能够看出，这段背景背景信息能够被模型学习并以此进行特定问题的回答。这其实就是一种非常简单的围绕本地知识进行问答的实现形式，不过需要注意的是，system role输入的信息也算是输入给模型的信息，因此受限于大语言模型的最大输入信息长度，单独借助system role在ChatCompletion.create函数中输入背景信息并不能真正意义上实现高度定制化、超大规模文本的本地知识库问答。但是，如果围绕着超大规模本地文本知识库先进行基于滑动窗口的文本切分，以确保切分后的小文本段落满足Max tokens要求，并且配合Embedding过程进行user问题和短文本的实时匹配，再把每个user问题匹配的关联度最高的文本以system消息的形式输入到模型中，再进行回答，则可以非常高效并且准确的实现本地知识库问答。而在这个过程中，借助system role进行背景文字的输入就非常基本的技术手段。

> 课程中将详细介绍上述本地知识库问答的高效实现方法，相关内容将在Embedding模型内容结束后进行介绍

- 借助.append方法进行多轮对话

&emsp;&emsp;最后，除了上述通过内部参数修改来实现不同功能外，messages参数的另一个重要应用是借助append方法来高效实现多轮对话。不同于Completion模型需要将历史问答都拼接为一个字符串并输入到新的prompt中来实现历史消息的输入，对于Chat模型来说，我们只需要将模型返回的message消息+用户新的提问message拼接到模型的messages参数中，并再次向模型进行提问，即可非常便捷的实现多轮对话。

首先我们可以使用to_dict()方法将模型返回的message信息转化为字典类型：

In [6]:
response.choices[0].message

<OpenAIObject at 0x1f0e62fc4f0> JSON: {
  "content": "\u9648\u660e\u4e8e1973\u5e74\u51fa\u751f\u3002",
  "role": "assistant"
}

In [7]:
type(response.choices[0].message)

openai.openai_object.OpenAIObject

In [8]:
response.choices[0].message.to_dict()

{'role': 'assistant', 'content': '陈明于1973年出生。'}

然后单独设置messages参数，并将此前的问题+答案进行拼接：

In [9]:
messages=[
    {"role": "system", "content": text},
    {"role": "user", "content": '请问陈明是那一年出生？'}
]

In [10]:
messages.append(response.choices[0].message.to_dict())

In [11]:
messages

[{'role': 'system',
  'content': '陈明，男，1973年7月15日出生于中国福建省厦门市。        1991年毕业于厦门大学电子科学与技术系，继而于1998年在北京大学获得信息技术博士学位。        毕业后的陈明在硅谷的一家著名科技公司工作了五年，专注于人工智能和机器学习的研发。'},
 {'role': 'user', 'content': '请问陈明是那一年出生？'},
 {'role': 'assistant', 'content': '陈明于1973年出生。'}]

此时messages参数就包含了最开始的问题+问题答案。接下来我们在messages消息中添加下一个问题：

In [12]:
messages.append({'role': 'user', 'content': '请问我刚才的问题是？'})

In [14]:
messages

[{'role': 'system',
  'content': '陈明，男，1973年7月15日出生于中国福建省厦门市。        1991年毕业于厦门大学电子科学与技术系，继而于1998年在北京大学获得信息技术博士学位。        毕业后的陈明在硅谷的一家著名科技公司工作了五年，专注于人工智能和机器学习的研发。'},
 {'role': 'user', 'content': '请问陈明是那一年出生？'},
 {'role': 'assistant', 'content': '陈明于1973年出生。'},
 {'role': 'user', 'content': '请问我刚才的问题是？'}]

接下来再次调用模型，并输入messages作为参数，此时模型将同时结合此前的所有消息，并围绕最后一个user信息进行回答：

In [15]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages
)

In [16]:
response.choices[0].message['content']

'您刚才的问题是“请问陈明是那一年出生？”'

能够发现，相比Completions模型，Chat类模型能够更加便捷的实现多轮对话。

- 代码补全

&emsp;&emsp;对于Chat模型来说，还有一个非常有趣且十分具有潜力的应用方向，那就是代码补全。通过在system message中设置好代码补全功能，然后在user message中著名代码实现的功能，例如计算斐波那契数列，则assistant message就能自动将后续代码补全：

In [25]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": '请将用户提出的python代码进行补全。'},
    {"role": "user", "content": 'def fibonacci(num):'}
  ]
)

In [26]:
response.choices[0].message['content']

'def fibonacci(num):\n    if num <= 0:\n        return []\n    elif num == 1:\n        return [0]\n    elif num == 2:\n        return [0, 1]\n    else:\n        fib = [0, 1]\n        for i in range(2, num):\n            fib.append(fib[i-1] + fib[i-2])\n        return fib'

将上述字符串形式展示的代码转化为python格式，得到完整代码如下：

In [28]:
def fibonacci(num):
    if num <= 0:
        return []
    elif num == 1:
        return [0]
    elif num == 2:
        return [0, 1]
    else:
        fib = [0, 1]
        for i in range(2, num):
            fib.append(fib[i-1] + fib[i-2])
        return fib

测试能否进行斐波那契数列计算：

In [29]:
fibonacci(5)

[0, 1, 1, 2, 3]

In [30]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": '什么是斐波那契数列？'}
  ]
)
response.choices[0].message['content']

'斐波那契数列是一个数列，其中每个数字都是前面两个数字的和。数列的前两个数字通常定义为0和1。因此，斐波那契数列的开始部分是：0，1，1，2，3，5，8，13，21，34，55，89，144，... 以此类推。斐波那契数列在数学和计算机科学中起着重要的作用，被广泛应用于算法设计、动态规划、递归等领域。'

能够发现，模型能够很好的完成斐波那契函数编写。

&emsp;&emsp;而若要深究模型为何能够完成斐波那契函数的代码补全，非常根本的原因在于模型对字符串'def fibonacci(num):'对应的含义非常了解——即本身知道斐波那契数列、也十分清楚斐波那契数列的这一特定的python函数编写方法，因此我们可以通过上述提示在Zero-shot的情况下就引导模型编写了斐波那契函数。而如果是其他类型的函数，其功能和实现方法无法被模型“天然的理解”（不在训练语料中），那么则需要更多的描述和提示，才能够引导模型完成函数创建。

=====================

<font size=4>更多完整版内容，见《大模型与AIGC技术实战》正课<br>添加微信：**littlecat_1207** 回复“<font color="red">**大模型**</font>”咨询正课详情哈</font>
